Example 1

In [ ]:
#https://www.programmersought.com/article/79252859610/
import matplotlib.pyplot as mp, numpy as np

# Primitive
origin = lambda x: 2 * x - x ** 2
x = np.linspace(0, 2, 9999)
mp.plot(x, origin(x), c='black')  # Visualization

# Derivative of the original function
derivative = lambda x: 2 - 2 * x

# Gradient rising demand
extreme_point = 0  # Initial value
alpha = 0.1  # Step, that is the learning rate
precision = 0.001  # Range of tolerance

while True:
    mp.scatter(extreme_point, origin(extreme_point))  # Visualization
    error = alpha * derivative(extreme_point)  # Climbing pace
    extreme_point += error  #Climbing #
    if abs(error) < precision:
        break  # Exit iterative error is small
mp.show()

Example 2

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

X, y = make_classification(1000, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, flip_y=0.05, random_state=4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

np.random.seed(4)
W = np.random.randn(2)

def predict(x, w):
    return 1 / (1 + np.exp(-np.matmul(x, w)))

def estimate_gradient(x, y, w):
    return np.nan_to_num((2 * y - 1) / predict(x, W) * x * np.nan_to_num(np.exp(-np.matmul(x, w))) / (np.nan_to_num(np.exp(-np.matmul(x, w))) + 1)**2)

a = 0.005 #learning rate
num_epochs = 45 #numbver of iteration

predictions = predict(X_train, W)
predictions = predictions.astype(int)

parameter_values = np.reshape(W, (1, -1))
accuracy_ratios = [accuracy_score(y_train, predictions)]
for i in range(num_epochs):
    for j in range(X_train.shape[0]):
        W = W + a * estimate_gradient(X_train[j], y_train[j], W)
    parameter_values = np.concatenate((parameter_values, np.reshape(W, (1, -1))), axis=0)
    predictions = predict(X_train, W)
    predictions = predictions.astype(int)
    accuracy_ratios.append(accuracy_score(y_train, predictions))

#training
predictions = predict(X_train, W)
predictions = predictions.astype(int)
print("Training: ", accuracy_score(y_train, predictions))

#testing
predictions = predict(X_test, W)
predictions = predictions.astype(int)
print("Testing: ", accuracy_score(y_test, predictions))

Example 3

In [ ]:
import numpy as np
import re
from pandas import DataFrame
import time as time
import matplotlib.pyplot as plt
import math
import csv

def get_data(filename):  
    with open(filename, 'r') as f:
        data = list(csv.reader(f, delimiter=","))

    data = np.array(data, dtype=np.float)
    df = DataFrame(data, columns=['x0','x1','x2','label'])

    dataMat = df.loc[:,['x0','x1','x2']]
    labelMat = df.loc[:,['label']]
    return dataMat,labelMat
    
def gradAscent(dataMat,labelMat,alpha = 0.1,maxstep = 1000): #Batch gradient ascent
    start_time = time.time()
    m,n = dataMat.shape
    weights = np.ones((n,1))
    labelMat = labelMat.values.reshape((16,1)) 
    for i in range(maxstep):
        h = sigmoid(np.dot(dataMat,weights).astype('int64'))  #Here is the matrix operation directly
        #labelMat = labelMat.values.reshape((30,1))                  #label is a one-dimensional, converted to two-dimensional
        error = labelMat-h                                    #Batch calculation error
        weights = weights + alpha*np.dot(dataMat.T,error)     #Update weight
        
    print("x1*", weights[0], " + x2*", weights[1], " + x3*", weights[2])
    duration = time.time()-start_time
    print('time:',duration)
    return weights

def stocGradAscent(dataMat,labelMat,alpha = 0.01):   #Random gradient ascent
    start_time = time.time()                         #Record program start time
    m,n = dataMat.shape
    weights = np.ones((n,1))                         #Allocation weight is 1
    
    for i in range(m):
        h = sigmoid(np.dot(dataMat.iloc[i],weights).astype('int64')) #Note: The dtype obtained from the inner product of the two two-dimensional arrays here is object, which needs to be converted to int64
        error = labelMat.iloc[i]-h                        #error
        weights = weights + alpha*dataMat.iloc[i].values.reshape((3,1))*error[0] #Update weight
    duration = time.time()-start_time
    print('time:',duration)
    return weights

def betterStoGradAscent(dataMat,labelMat,alpha = 0.01,maxstep = 150):
    start_time = time.time()
    m,n = dataMat.shape
    weights = np.ones((n,1))
    for j in range(maxstep):
        for i in range(m):
            alpha = 4/(1+i+j) + 0.01                         #Set the update rate to decrease with iteration
            h = sigmoid(np.dot(dataMat.iloc[i],weights).astype('int64'))
            error = labelMat.iloc[i]-h
            weights = weights + alpha*dataMat.iloc[i].values.reshape((3,1))*error[0]
    duration = time.time()-start_time
    print('time:',duration)
    return weights

def sigmoid(x):
    return 1.0/(1+np.exp(-x))

def show(dataMat, labelMat, weights):
    #dataMat = np.mat(dataMat)
    #labelMat = np.mat(labelMat)
    m,n = dataMat.shape
    min_x = min(dataMat.iloc[:, 1])
    max_x = max(dataMat.iloc[:, 1])
    xcoord1 = []; ycoord1 = []
    xcoord2 = []; ycoord2 = []
    for i in range(m):
        if int(labelMat.iloc[i]) == 0:            
            xcoord1.append(dataMat.iloc[i, 1]); ycoord1.append(dataMat.iloc[i, 2])
        elif int(labelMat.iloc[i]) == 1:
            xcoord2.append(dataMat.iloc[i, 1]); ycoord2.append(dataMat.iloc[i, 2])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(xcoord1, ycoord1, s=30, c="red", marker="s")
    ax.scatter(xcoord2, ycoord2, s=30, c="green")
    x = np.arange(min_x, max_x, 0.1)
    y = (-float(weights[0]) - float(weights[1])*x) / float(weights[2])
    ax.plot(x, y)
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.show()

if __name__=='__main__':
    dataMat,labelMat = get_data('data2.csv')
    #weights = gradAscent(dataMat,labelMat)
    #weights = stocGradAscent(dataMat,labelMat)
    weights = betterStoGradAscent(dataMat,labelMat)
    show(dataMat,labelMat,weights)


Example 4

In [ ]:
import numpy as np
import pandas as pd
from math import exp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#read the data
data = pd.read_csv("divorce.csv", delimiter = ';')

df = pd.DataFrame(data)
value = [1]*170
#inserting a bias 
df = df.insert(0,"Atr0",value,False)

x = data.iloc[: , :-1]
y = data.iloc[: , -1]
x = np.array(x)
y = np.array(y)

#Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.15, random_state=0) 

class MyLogisticRegression:
    def __init__(self, learning_rate=0.01, max_iterations=1000):
        '''Initialize variables
        Args:
            learning_rate  : Learning Rate
            max_iterations : Max iterations for training weights
        '''
        # Initialising all the parameters
        self.learning_rate  = learning_rate
        self.max_iterations = max_iterations
        self.likelihoods    = []
        
        # Define epsilon because log(0) is not defined
        self.eps = 1e-7

    def sigmoid(self, z):
        '''Sigmoid function: f:R->(0,1)
        Args:
            z : A numpy array (num_samples,)
        Returns:
            A numpy array where sigmoid function applied to every element
        '''
        ### START CODE HERE
        sig_z = (1/(1+np.exp(-z)))
        ### END CODE HERE
        
        assert (z.shape==sig_z.shape), 'Error in sigmoid implementation. Check carefully'
        return sig_z
    
    def log_likelihood(self, y_true, y_pred):
        '''Calculates maximum likelihood estimate
        Remember: y * log(yh) + (1-y) * log(1-yh)
        Note: Likelihood is defined for multiple classes as well, but for this dataset
        we only need to worry about binary/bernoulli likelihood function
        Args:
            y_true : Numpy array of actual truth values (num_samples,)
            y_pred : Numpy array of predicted values (num_samples,)
        Returns:
            Log-likelihood, scalar value
        '''
        # Fix 0/1 values in y_pred so that log is not undefined
        y_pred = np.maximum(np.full(y_pred.shape, self.eps), np.minimum(np.full(y_pred.shape, 1-self.eps), y_pred))
        
        ### START CODE HERE
        likelihood = (y_true*np.log(y_pred)+(1-y_true)*np.log(1-y_pred))
    
        ### END CODE HERE
        
        return np.mean(likelihood)
    
    def fit(self, X, y):
        '''Trains logistic regression model using gradient ascent
        to gain maximum likelihood on the training data
        Args:
            X : Numpy array (num_examples, num_features)
            y : Numpy array (num_examples, )
        Returns: VOID
        '''
        
        num_examples = X.shape[0]
        num_features = X.shape[1]
        
        ### START CODE HERE
        
        # Initialize weights with appropriate shape
        self.weights = np.zeros((X.shape[1]))
        # print("Z",self.weights.shape)
        # print(X.shape)
        
        
        # Perform gradient ascent
        for i in range(self.max_iterations):
            # Define the linear hypothesis(z) first
            # HINT: what is our hypothesis function in linear regression, remember?
            
            z  = np.dot(X,self.weights)
          
            # Output probability value by appplying sigmoid on z
            y_pred = self.sigmoid(z)
            
            # Calculate the gradient values
            # This is just vectorized efficient way of implementing gradient. 
            gradient = np.mean((y-y_pred)*X.T, axis=1)
            
            # Update the weights
            # Caution: It is gradient ASCENT not descent
            self.weights +=  self.learning_rate*gradient
            
            # Calculating log likelihood
            likelihood = self.log_likelihood(y,y_pred)

            self.likelihoods.append(likelihood)
    
        ### END CODE HERE
    
    def predict_proba(self,X):
        '''Predict probabilities for given X.
        Remember sigmoid returns value between 0 and 1.
        Args:
            X : Numpy array (num_samples, num_features)
        Returns:
            probabilities: Numpy array (num_samples,)
        '''
        if self.weights is None:
            raise Exception("Fit the model before prediction")
      
        ### START CODE HERE
               
        z = np.dot(X,self.weights)
        probabilities = self.sigmoid(z)
        # probabilities.reshape(probabilities.shape[0],1)
        
        ### END CODE HERE
        
        return probabilities
    
    def predict(self, X, threshold=0.5):
        '''Predict/Classify X in classes
        Args:
            X         : Numpy array (num_samples, num_features)
            threshold : scalar value above which prediction is 1 else 0
        Returns:
            binary_predictions : Numpy array (num_samples,)
        '''
        # Thresholding probability to predict binary values
        
        binary_predictions = np.array(list(map(lambda x: 1 if x>threshold else 0, self.predict_proba(X))))
        
        return binary_predictions

#create learning model
model = MyLogisticRegression()
model.fit(x_train,y_train)

#prediction
y_pred = model.predict(x_test)

def accuracy(y_true,y_pred):
    '''Compute accuracy.
    Accuracy = (Correct prediction / number of samples)
    Args:
        y_true : Truth binary values (num_examples, )
        y_pred : Predicted binary values (num_examples, )
    Returns:
        accuracy: scalar value
    '''
    count=0
    ### START CODE HERE
    y_cap = y_pred - y_true
    count = np.count_nonzero(y_cap==0)   
    accuracy = (count/len(y_pred))*100
    ### END CODE HERE
    return accuracy

#get the accuracy
accuracy(y_test,y_pred)

plt.plot([i+1 for i in range(len(model.likelihoods))], model.likelihoods)
plt.title("Log-Likelihood curve")
plt.xlabel("Iteration num")
plt.ylabel("Log-likelihood")
plt.show()